This notebook is used to experiement with our customizable model. We train an instance of it, and then later we perform a hyperparameter sweep to find optimal hyperparameter values using the optuna library.

In [6]:
# imports
import argparse
from argparse import Namespace

from pytorch_lightning import Trainer, LightningModule, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger
from torchsummary import summary

from yeastdnnexplorer.data_loaders.synthetic_data_loader import SyntheticDataLoader
from yeastdnnexplorer.ml_models.simple_model import SimpleModel
from yeastdnnexplorer.ml_models.customizable_model import CustomizableModel

import optuna

import matplotlib.pyplot as plt
import seaborn as sns

from yeastdnnexplorer.probability_models.generate_data import (
    perturbation_effect_adjustment_function_with_tf_relationships,
)

seed_everything(42)

Seed set to 42


42

Defining checkpoints and loggers for the model. Checkpoints tell pytorch when to save instances of the model (that can be loaded and inspected later) and loggers tell pytorch how to format the metrics that the model logs during its training.

In [7]:
# define checkpoints for the model
# tells it when to save snapshots of the model during training
# Callback to save the best model based on validation loss

# this one works for simple model
# best_model_checkpoint = ModelCheckpoint(
#     monitor="val_loss",
#     mode="min",
#     filename="best-model-{epoch:02d}-{val_loss:.2f}",
#     save_top_k=1,
# )

best_model_checkpoint = ModelCheckpoint(
    monitor="val_mse",
    mode="min",
    filename="best-model-{epoch:02d}-{val_loss:.2f}.ckpt",
    save_top_k=1,
)

# Callback to save checkpoints every 2 epochs, regardless of performance
periodic_checkpoint = ModelCheckpoint(
    filename="periodic-{epoch:02d}.ckpt",
    every_n_epochs=2,
    save_top_k=-1,  # Setting -1 saves all checkpoints
)

# define loggers for the model
tb_logger = TensorBoardLogger("logs/tensorboard_logs")
csv_logger = CSVLogger("logs/csv_logs")

Training an instance of the customizable model. We are using perturbation_effect_adjustment_function_with_tf_relationships to adjust the means of the generated data based on which combinations of transcription factors are bound to the gene in question.  

In [3]:
tf_relationships_dict = {
    0: [2, 4, 7],
    1: [8],
    2: [3, 9],
    3: [1, 6],
    4: [5],
    5: [0, 2, 8],
    6: [4],
    7: [1, 4],
    8: [6],
    9: [0, 3, 8],
}

data_module = SyntheticDataLoader(
    batch_size=32,
    num_genes=4000,
    signal_mean=3.0,
    signal=[0.5] * 10,  # old: [0.1, 0.15, 0.2, 0.25, 0.3],
    n_sample=[1, 2, 2, 4, 4],  # sum of this is num of tfs
    val_size=0.1,
    test_size=0.1,
    random_state=42,
    max_mean_adjustment=3.0,
    adjustment_function=perturbation_effect_adjustment_function_with_tf_relationships,
    tf_relationships=tf_relationships_dict,
)

num_tfs = sum(data_module.n_sample)  # sum of all n_sample is the number of TFs

model = CustomizableModel(
    input_dim=num_tfs,
    output_dim=num_tfs,
    lr=0.01,
    hidden_layer_num=3,
    hidden_layer_sizes=[128, 64, 32],
    activation="ReLU",
    optimizer="Adam",
    L2_regularization_term=0.0,
    dropout_rate=0.0,
)

trainer = Trainer(
    max_epochs=10,
    deterministic=True,
    accelerator="cpu",
    callbacks=[best_model_checkpoint, periodic_checkpoint],
    logger=[tb_logger, csv_logger],
)

trainer.fit(model, data_module)

test_results = trainer.test(model, datamodule=data_module)
print("Printing test results...")
print(test_results)  # this prints all metrics that were logged during the test phase

# print summary of model
print("Printing model summary...")
summary(model, (num_tfs, num_tfs))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:250: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, Y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:253: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val, Y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(
/Users/benmueller/2024Classes/BrentResearch/git

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_mse            1.7528094053268433
       test_nrmse           0.20540115237236023
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Printing test results...
[{'test_mse': 1.7528094053268433, 'test_nrmse': 0.20540115237236023}]
Printing model summary...
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 13, 128]           1,792
              ReLU-2              [-1, 13, 128]               0
           Dropout-3              [-1, 13, 128]               0
            Linear-4               [-1, 13, 64]         

Here we define a TF relationships dictionary to be used while training models in our hyperparameter sweep. It is important that we use the same relationships for each of the models we train since the only things being varied in the hyperparameter sweep should be the hyperparameters themselves.

In [8]:
tf_relationships_dict = {
    0: [2, 4, 7],
    1: [8],
    2: [3, 9],
    3: [1, 6],
    4: [5],
    5: [0, 2, 8],
    6: [4],
    7: [1, 4],
    8: [6],
    9: [0, 3, 8],
}

To perform our hyperparameter sweep using the optuna library, we need to define an objective function that reutnrs a scalar value. We then minimize this obective function during the hyperparamter sweep.

In [9]:
def objective(trial):
    # model hyperparameters
    lr = trial.suggest_categorical("lr", [0.01])
    hidden_layer_num = trial.suggest_categorical("hidden_layer_num", [1, 2, 3, 5])
    activation = trial.suggest_categorical(
        "activation", ["ReLU", "Sigmoid", "Tanh", "LeakyReLU"]
    )
    optimizer = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    L2_regularization_term = trial.suggest_categorical(
        "L2_regularization_term", [0.0, 0.1]
    )
    dropout_rate = trial.suggest_categorical(
        "dropout_rate", [0.0, 0.5]
    )

    # data module hyperparameters
    batch_size = trial.suggest_categorical("batch_size", [32])

    # training hyperparameters
    max_epochs = trial.suggest_categorical(
        "max_epochs", [10]
    ) # default is 10

    # defining what to pass in for the hidden layer sizes list based on the number of hidden layers
    hidden_layer_sizes_configurations = {
        1: [[64], [256]],
        2: [[64, 32], [256, 64]],
        3: [[256, 128, 32], [512, 256, 64]],
        5: [[512, 256, 128, 64, 32]],
    }
    hidden_layer_sizes = trial.suggest_categorical(
        f"hidden_layer_sizes_{hidden_layer_num}_layers",
        hidden_layer_sizes_configurations[hidden_layer_num],
    )

    print("=" * 70)
    print("About to create model with the following hyperparameters:")
    print(f"lr: {lr}")
    print(f"hidden_layer_num: {hidden_layer_num}")
    print(f"hidden_layer_sizes: {hidden_layer_sizes}")
    print(f"activation: {activation}")
    print(f"optimizer: {optimizer}")
    print(f"L2_regularization_term: {L2_regularization_term}")
    print(f"dropout_rate: {dropout_rate}")
    print(f"batch_size: {batch_size}")
    print(f"max_epochs: {max_epochs}")
    print("")

    # create data module
    data_module = SyntheticDataLoader(
        batch_size=batch_size,
        num_genes=4000,
        signal_mean=3.0,
        signal=[0.5] * 10,  # old: [0.1, 0.15, 0.2, 0.25, 0.3],
        n_sample=[1, 2, 2, 4, 4],  # sum of this is num of tfs
        val_size=0.1,
        test_size=0.1,
        random_state=42,
        max_mean_adjustment=3.0,
        adjustment_function=perturbation_effect_adjustment_function_with_tf_relationships,
        tf_relationships=tf_relationships_dict,
    )

    num_tfs = sum(data_module.n_sample)  # sum of all n_sample is the number of TFs

    # create model
    model = CustomizableModel(
        input_dim=num_tfs,
        output_dim=num_tfs,
        lr=lr,
        hidden_layer_num=hidden_layer_num,
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        optimizer=optimizer,
        L2_regularization_term=L2_regularization_term,
        dropout_rate=dropout_rate,
    )

    # create trainer
    trainer = Trainer(
        max_epochs=max_epochs,
        deterministic=True,
        accelerator="cpu",
        callbacks=[
            best_model_checkpoint
        ],  # not using periodic checkpoint as that would be way too many checkpoints, can add back if we choose a specific hyperparam config that we want to look more closely at
        logger=[tb_logger, csv_logger],
    )

    # train model
    trainer.fit(model, data_module)

    # get best validation loss from the model
    return trainer.callback_metrics["val_mse"]

Now we can run the hyperparater sweep and print out what we found to be the optimal set of hyperparaters. Note that this will create a very large amount of output since we are training an instance of the model for every possible combination of hyperparameters. We output the best combination of hyperparameters along with the mse of the model that used those hyperparameters.

In [15]:
# Perform hyperparameter optimization using Optuna
study = optuna.create_study(
    direction="minimize",
    storage="sqlite:///db.sqlite3",
    study_name="LargeSweepForCustomizableModelWithDependentTFRelationships"
)
study.optimize(objective, n_trials=75)

# Get the best hyperparameters and their corresponding values
best_params = study.best_params
best_loss = study.best_value

print("\n" * 5)
print("RESULTS" + ("=" * 70))
print(f"Best hyperparameters: {best_params}")
print(f"Best loss: {best_loss}")

[I 2024-04-09 09:11:55,587] A new study created in RDB with name: LargeSweepForCustomizableModelWithDependentTFRelationships
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/.venv/lib/python3.11/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256, 128, 32] which is of type list.
  warnings.warn(message)
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/.venv/lib/python3.11/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 256, 64] which is of type list.
  warnings.warn(message)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


About to create model with the following hyperparameters:
lr: 0.01
hidden_layer_num: 3
hidden_layer_sizes: [256, 128, 32]
activation: Tanh
optimizer: RMSprop
L2_regularization_term: 0.1
dropout_rate: 0.0
batch_size: 32
max_epochs: 10



/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:250: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, Y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:253: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val, Y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:256: UserWarning: To copy construct from a tensor, it is recommended

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
[I 2024-04-09 09:50:16,729] Trial 74 finished with value: 1.6040574312210083 and parameters: {'lr': 0.01, 'hidden_layer_num': 3, 'activation': 'Sigmoid', 'optimizer': 'RMSprop', 'L2_regularization_term': 0.0, 'dropout_rate': 0.5, 'batch_size': 32, 'max_epochs': 10, 'hidden_layer_sizes_3_layers': [256, 128, 32]}. Best is trial 7 with value: 1.5285433530807495.








RESULTS======================================================================
Best hyperparameters: {'lr': 0.01, 'hidden_layer_num': 2, 'activation': 'LeakyReLU', 'optimizer': 'RMSprop', 'L2_regularization_term': 0.0, 'dropout_rate': 0.0, 'batch_size': 32, 'max_epochs': 10, 'hidden_layer_sizes_2_layers': [64, 32]}
Best loss: 1.5285433530807495


In [19]:
print("\n" * 5)
print("RESULTS" + ("=" * 70))
print(f"Best hyperparameters: {best_params}")
print(f"Best loss: {best_loss}")







RESULTS======================================================================
Best hyperparameters: {'lr': 0.01, 'hidden_layer_num': 2, 'activation': 'LeakyReLU', 'optimizer': 'RMSprop', 'L2_regularization_term': 0.0, 'dropout_rate': 0.0, 'batch_size': 32, 'max_epochs': 10, 'hidden_layer_sizes_2_layers': [64, 32]}
Best loss: 1.5285433530807495


Now that we know what the best hyperparameters are for the probelem at hand, we can train a model with these parameter for a large amount of epochs

In [10]:
tf_relationships_dict = {
    0: [2, 4, 7],
    1: [8],
    2: [3, 9],
    3: [1, 6],
    4: [5],
    5: [0, 2, 8],
    6: [4],
    7: [1, 4],
    8: [6],
    9: [0, 3, 8],
}

data_module = SyntheticDataLoader(
    batch_size=32,
    num_genes=4000,
    signal_mean=3.0,
    signal=[0.5] * 10,  # old: [0.1, 0.15, 0.2, 0.25, 0.3],
    n_sample=[1, 2, 2, 4, 4],  # sum of this is num of tfs
    val_size=0.1,
    test_size=0.1,
    random_state=42,
    max_mean_adjustment=3.0,
    adjustment_function=perturbation_effect_adjustment_function_with_tf_relationships,
    tf_relationships=tf_relationships_dict,
)

num_tfs = sum(data_module.n_sample)  # sum of all n_sample is the number of TFs

model = CustomizableModel(
    input_dim=num_tfs,
    output_dim=num_tfs,
    lr=0.01,
    hidden_layer_num=2,
    hidden_layer_sizes=[64, 32],
    activation="LeakyReLU",
    optimizer="RMSprop",
    L2_regularization_term=0.0,
    dropout_rate=0.0,
)

trainer = Trainer(
    max_epochs=50,
    deterministic=True,
    accelerator="cpu",
    callbacks=[best_model_checkpoint, periodic_checkpoint],
    logger=[tb_logger, csv_logger],
)

trainer.fit(model, data_module)

test_results = trainer.test(model, datamodule=data_module)
print("Printing test results...")
print(test_results)  # this prints all metrics that were logged during the test phase

# print summary of model
print("Printing model summary...")
summary(model, (num_tfs, num_tfs))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type       | Params
---------------------------------------------
0 | r2            | R2Score    | 0     
1 | activation    | LeakyReLU  | 0     
2 | input_layer   | Linear     | 896   
3 | hidden_layers | ModuleList | 2.1 K 
4 | output_layer  | Linear     | 429   
5 | dropout       | Dropout    | 0     
---------------------------------------------
3.4 K     Trainable params
0         Non-trainable params
3.4 K     Total params
0.014     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_mse            1.6123228073120117
       test_nrmse           0.2009783387184143
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Printing test results...
[{'test_mse': 1.6123228073120117, 'test_nrmse': 0.2009783387184143}]
Printing model summary...
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 13, 64]             896
         LeakyReLU-2               [-1, 13, 64]               0
           Dropout-3               [-1, 13, 64]               0
            Linear-4               [-1, 13, 32]           